In [87]:
import pandas as pd 
import catboost as cb 
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
from sklearn.metrics import f1_score

In [66]:
import warnings
warnings.filterwarnings('ignore')

In [67]:
train_values = pd.read_csv('train_values.csv', index_col='building_id')
train_values1 = pd.read_csv('train_values_short1.csv', index_col='building_id')
train_values_baseline = pd.get_dummies(train_values)
train_values_short1 = train_values1.merge(train_values[["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"]],
                                            left_index=True, right_index=True)
train_values_catboost = pd.read_csv("train_values_catboost.csv", index_col='building_id')
train_values_lgbm = pd.read_csv("train_values_lightGBM.csv", index_col='building_id')

train_labels = pd.read_csv('train_labels.csv', index_col='building_id')
le = LabelEncoder()
train_labels_encoded = le.fit_transform(y=train_labels.values.ravel())

In [68]:
categoricas = []
for col in train_values_catboost.columns:
    if ((col != 'min_geo_id') & (col != 'max_geo_id')):
        categoricas.append(col+'_cat')
        train_values_catboost[col] = train_values_catboost[col].astype("category")
        # Las renombro porque tiene las mismas columnas que los otros pero son disntintas
        train_values_catboost.rename(columns={col:col+'_cat'}, inplace=True)

In [69]:
cols_baseline = train_values_baseline.columns
cols_short1 = train_values_short1.columns
cols_catboost = train_values_catboost.columns
cols_lgbm = train_values_lgbm.columns

In [70]:
# Guardo todas las columnas en el mismo DF
train_values_baseline.loc[:,cols_short1] = train_values_short1
train_values_baseline.loc[:,cols_catboost] = train_values_catboost
train_values_baseline.loc[:, cols_lgbm] = train_values_lgbm
train_values = train_values_baseline

In [88]:
class EncontradorPesos(ClassifierMixin):
    def __init__(self, peso_xgb_baseline, peso_xgb_short1, peso_catboost, peso_lgbm, categoricas_catboost):
        
        self.peso_xgb_baseline = peso_xgb_baseline
        self.xgb_baseline = xgb_baseline = XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 273,
                                                         verbosity=0, use_label_encoder=False)
        
        self.peso_xgb_short1 = peso_xgb_short1
        self.xgb_short1=XGBClassifier( objective = "multi:softmax", use_label_encoder=False, seed=30,
                         max_depth=9,
                         min_child_weight=6,
                         gamma=0.2554138025988315,
                        colsample_bytree = 0.7773754946860542,
                        subsample = 0.9030471581301206,
                         learning_rate =0.1, 
                         n_estimators=273,
                        verbosity=0
                         )
        
        self.peso_catboost = peso_catboost
        self.catboost = cb.CatBoostClassifier(loss_function='MultiClass', eval_metric='TotalF1:average=Micro', 
                                              iterations=100, verbose=False)
        self.categoricas_catboost = categoricas_catboost
        
        self.lgbm = LGBMClassifier(random_state=2021, num_iterations= 273,
                                   num_leaves= 70, objective= 'regression', colsample_bytree=0.77)
        self.peso_lgbm = peso_lgbm
    
                 
    def fit(self, X, y):
        self.xgb_short1.fit(X[cols_short1], y)
        self.xgb_baseline.fit(X[cols_baseline], y)
        self.catboost.fit(X[cols_catboost], y, self.categoricas_catboost)
        self.lgbm.fit(X[cols_lgbm],y)
        
    def predict(self, X):
        probas_xgb_short1 =  self.xgb_short1.predict_proba(X[cols_short1]) * self.peso_xgb_short1
        probas_xgb_baseline =  self.xgb_baseline.predict_proba(X[cols_baseline]) * self.peso_xgb_baseline
        probas_catboost =  self.catboost.predict_proba(X[cols_catboost]) * self.peso_catboost
        probas_lgbm =  self.lgbm.predict_proba(X[cols_lgbm]) * self.peso_lgbm

        
        preds = []
        for i in range(len(X[cols_short1])):
            proba_grado1 = probas_xgb_short1[i][0] + probas_xgb_baseline[i][0] \
                           + probas_catboost[i][0] + probas_lgbm[i][0]
            proba_grado2 = probas_xgb_short1[i][1] + probas_xgb_baseline[i][1] \
                           + probas_catboost[i][1] + probas_lgbm[i][1]
            proba_grado3 = probas_xgb_short1[i][2] + probas_xgb_baseline[i][2] \
                           + probas_catboost[i][2] + probas_lgbm[i][2]
            
            
            max_proba = max([proba_grado1, proba_grado2, proba_grado3])
            preds.append([proba_grado1, proba_grado2, proba_grado3].index(max_proba))
        
 
        return preds

In [89]:
def funcion_a_optimizar(peso_baseline, peso_short1, peso_lgbm):

            model = EncontradorPesos(peso_xgb_baseline=peso_baseline,
                                     peso_xgb_short1=peso_short1, peso_catboost=1, peso_lgbm = peso_lgbm,
                        categoricas_catboost=categoricas)

            score = cross_val_score(model, train_values, train_labels_encoded, scoring='f1_micro', cv=8).mean()
            print("Score: " + str(score))
            print("\t Peso baseline: " + str(peso_baseline))
            print("\t Peso short1: " + str(peso_short1))
            print("\t Peso lgbm: " + str(peso_lgbm))
            print()
            
            return(score)

In [90]:
limites = {"peso_baseline":(4,5), "peso_short1":(3,4), "peso_lgbm":(1.5,2.5)}

In [91]:
optimizer = BayesianOptimization(
    f=funcion_a_optimizar,
    random_state=10,
    pbounds=limites,
)


logger = JSONLogger(path="./logs_ultima_carta.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [ ]:
optimizer.probe({'peso_baseline': 4.418722982360061,
  'peso_lgbm': 1.847955192063759,
  'peso_short1': 3.6931656858666138}, lazy=True)

In [16]:
optimizer.maximize(
    init_points=10,
    n_iter=3000,
)

Score: 0.7591200000000001
	 Peso baseline: 5.497607765077223
	 Peso short1: 3.4495932910616953
	 Peso lgbm: 1.3982084147631086

Score: 0.7597599999999999
	 Peso baseline: 4.396125729519248
	 Peso short1: 3.338221673125071
	 Peso lgbm: 1.7126368546387505

Score: 0.76
	 Peso baseline: 4.176679628348021
	 Peso short1: 4.906786692389873
	 Peso lgbm: 1.6224317820413567

Score: 0.7598199999999999
	 Peso baseline: 4.007896532655829
	 Peso short1: 4.625241923304227
	 Peso lgbm: 1.414630716062932

Score: 0.7593999999999999
	 Peso baseline: 5.225052133658776
	 Peso short1: 3.5837521363412663
	 Peso lgbm: 1.6661063809181593

Score: 0.75938
	 Peso baseline: 5.835548245025887
	 Peso short1: 4.085088736022523
	 Peso lgbm: 1.6574909400772286

Score: 0.76002
	 Peso baseline: 4.284340095203054
	 Peso short1: 4.348267230132691
	 Peso lgbm: 1.248008912061763

Score: 0.75952
	 Peso baseline: 4.883666348845992
	 Peso short1: 4.235533956938634
	 Peso lgbm: 1.3208167919999525

Score: 0.75964
	 Peso baseline:

KeyboardInterrupt: 

In [17]:
optimizer.max

{'target': 0.7604600000000001,
 'params': {'peso_baseline': 4.418722982360061,
  'peso_lgbm': 1.847955192063759,
  'peso_short1': 3.6931656858666138}}

In [18]:
def target(resultado):
    return resultado['target']

def mostrar(resultados, min=0):
    
    a_borrar = []
    for resultado in resultados:
        if(resultado['target']<min):
            a_borrar.append(resultado)
    resultados_importantes = [a for a in resultados if a not in a_borrar]
    
    resultados_importantes.sort(reverse=True, key=target)
    
    for resultado in resultados_importantes:
        print("target: " , resultado['target'])
        for key,value in resultado['params'].items():
            print("\t\t", key, ':', value)
        print()

In [ ]:
mostrar(optimizer.res)